In [1]:
import duckdb
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine

import pandas as pd
from typing import List, Dict, Any
from oaklib import get_adapter

what's a generic, high performance way to swap data between the duckdb and in-memory Python data structures?

wrap in sqlalchemy and use pandas?

I'm adding connections and transactions... seems like a lot of overhead

In [11]:
duckdb_filename = "ncbi_biosamples_dupe.duckdb"

In [12]:
def open_duckdb_file(filename):
    """
    Creates a file-based DuckDB database and returns the connection.
  
    Args:
      filename: The name of the DuckDB database file to create.
  
    Returns:
      duckdb.DuckDBPyConnection: The DuckDB connection object.
    """
    conn = duckdb.connect(database=filename)
    return conn

In [13]:
def create_engine_connection(db_path: str) -> Engine:
    """
    Creates and returns an SQLAlchemy Engine for the DuckDB database.

    Args:
        db_path (str): Path to the DuckDB file.

    Returns:
        sqlalchemy.engine.Engine: A connection engine for the DuckDB database.
    """
    try:
        engine = create_engine(f"duckdb:///{db_path}")
        return engine
    except Exception as e:
        raise RuntimeError(f"Failed to create SQLAlchemy engine: {e}")

In [14]:
def list_tables(engine: Engine) -> None:
    """
    Prints a list of all tables in the current schema.

    Args:
        engine (sqlalchemy.engine.Engine): SQLAlchemy engine for the database.
    """
    try:
        with engine.connect() as connection:
            result = connection.execute(text("SHOW TABLES"))
            print("Tables in the current schema:")
            for row in result:
                print(f" - {row[0]}")
    except Exception as e:
        raise RuntimeError(f"Error listing tables: {e}")

In [15]:

def extract_curies_from_text(
    text: str,
    row_id: int = None,  # Generic parameter for row context
    prefix_min_len: int = 3,
    prefix_max_len: int = 6,
    local_id_min_len: int = 7,
    local_id_max_len: int = 8,
    prefix_chars_allowed: str = r"[a-zA-Z]",
    local_id_chars_allowed: str = r"[0-9]",
    delimiter_chars_allowed: str = r"[_:]",
) -> List[dict]:
    """
    Extract ontology class CURIEs from text and return them as dictionaries.

    Args:
        text (str): The input text.
        row_id (int, optional): A generic ID or reference for the row context.
        prefix_min_len (int): Minimum length of the prefix part of the CURIE.
        prefix_max_len (int): Maximum length of the prefix part of the CURIE.
        local_id_min_len (int): Minimum length of the local ID part of the CURIE.
        local_id_max_len (int): Maximum length of the local ID part of the CURIE.
        prefix_chars_allowed (str): Allowed characters for the prefix.
        local_id_chars_allowed (str): Allowed characters for the local ID.
        delimiter_chars_allowed (str): Allowed delimiters between prefix and local ID.

    Returns:
        List[dict]: A list of dictionaries containing CURIE parts and row context.
    """
    import re

    pattern = rf"""
        \b                                      # Word boundary
        (?P<prefix>{prefix_chars_allowed}{{{prefix_min_len},{prefix_max_len}}})  # Prefix
        (?P<delimiter>{delimiter_chars_allowed})                               # Delimiter
        (?P<local_id>{local_id_chars_allowed}{{{local_id_min_len},{local_id_max_len}}})  # Local ID
        \b                                      # Word boundary
    """
    matches = re.finditer(pattern, text, re.VERBOSE)
    return [
        {
            "row_id": row_id,  # General ID reference
            "curie_prefix": match.group("prefix"),
            "curie_delimiter": match.group("delimiter"),
            "curie_local_id": match.group("local_id"),
        }
        for match in matches
    ]


In [16]:
def class_detection_by_label(
    text: str,
    ontology_adapters: Dict[str, Any],
    row_id: int = None,
    min_annotated_length: int = 3
) -> List[Dict[str, Any]]:
    """
    Detect ontology class labels in a string using multiple ontology adapters and collect the annotations.
    Only annotations with a match string length >= min_annotated_length are included.
    Additionally, a flag 'is_longest_match' is added to indicate the longest match for the string.

    Args:
        text (str): The input string to be analyzed.
        ontology_adapters (dict): Dictionary of ontology adapters.
        row_id (int, optional): A generic ID or reference for the row context.
        min_annotated_length (int): Minimum length for the annotated match string to be included.

    Returns:
        List[dict]: A list of dictionaries with the annotations, each representing a new row.
    """
    annotations_for_this_string = []

    # Annotate the string using each ontology adapter
    for _, adapter in ontology_adapters.items():
        annotations = adapter.annotate_text(text)

        if annotations:  # If there are annotations
            for annotation in annotations:
                subject_string = annotation.match_string

                # Only include annotations where subject_string length is >= min_annotated_length
                if len(subject_string) >= min_annotated_length:
                    # Build the annotation dictionary
                    annotations_dict = {
                        "id": row_id,
                        "subject_string": subject_string,
                        "subject_start": annotation.subject_start,
                        "subject_end": annotation.subject_end,
                        "predicate_id": annotation.predicate_id,
                        "concluded_curie": annotation.object_id,
                        "object_string": annotation.object_label,
                    }
                    annotations_for_this_string.append(annotations_dict)

    # Determine the longest match for this string
    if annotations_for_this_string:
        longest_annotation = max(annotations_for_this_string, key=lambda x: len(x['subject_string']))

        # Mark each annotation as the longest or not
        for annotation in annotations_for_this_string:
            annotation['is_longest_match'] = annotation['subject_string'] == longest_annotation['subject_string']

    return annotations_for_this_string


In [17]:
def create_ontology_adapters(ontology_short_names: list) -> dict:
    """
    Create a dictionary of OAK adapters for each ontology short name.

    Args:
        ontology_short_names (list): A list of ontology short names (e.g., ['envo', 'po']).

    Returns:
        dict: A dictionary where keys are ontology short names and values are the OAK adapters.
    """
    adapters = {}
    for short_name in ontology_short_names:
        adapter_string = f"sqlite:obo:{short_name}"  # Create the adapter string
        try:
            adapters[short_name] = get_adapter(adapter_string)  # Get the adapter and add to dictionary
        except Exception as e:
            print(f"Warning: Failed to create adapter for {short_name}. Error: {e}")
    return adapters

In [18]:
duckdb_engine = create_engine_connection(duckdb_filename)

In [19]:
list_tables(duckdb_engine)

Tables in the current schema:
 - attributes
 - contexts_to_normalized_strings
 - harmonized_attributes_wide
 - ids
 - links
 - normalized_context_strings
 - organism
 - overview


In [20]:
with duckdb_engine.connect() as connection:
    transaction = connection.begin()
    connection.execute(text("DROP TABLE IF EXISTS main.normalized_contexts;"))
    connection.execute(text("drop table if exists main.normalized_context_strings;"))
    connection.execute(text("DROP TABLE IF EXISTS normalized_contexts;"))
    connection.execute(text("drop table if exists normalized_context_strings;"))
    connection.execute(text("""
    drop table if exists main.contexts_to_normalized_strings;
    """))
    connection.execute(text("""
    drop table if exists contexts_to_normalized_strings;
    """))
    transaction.commit()

In [21]:
list_tables(duckdb_engine)

Tables in the current schema:
 - attributes
 - harmonized_attributes_wide
 - ids
 - links
 - organism
 - overview


In [22]:
with duckdb_engine.connect() as connection:
    
    transaction = connection.begin()
    
    connection.execute(text("""
    CREATE TEMPORARY TABLE normalized_contexts AS
    SELECT
        id,
        harmonized_name,
        content,
        regexp_replace(trim(lower(content)), '\\s+', ' ', 'g') AS normalized
    FROM
        main.attributes
    WHERE
        harmonized_name IN ('env_broad_scale', 'env_local_scale', 'env_medium');
    """))

    connection.execute(text("""
    CREATE TABLE main.normalized_context_strings (
    normalized_context_string_id INTEGER PRIMARY KEY,
    normalized_context_string TEXT UNIQUE
    );
    """))

    connection.execute(text("""
    INSERT INTO main.normalized_context_strings (normalized_context_string_id, normalized_context_string)
    SELECT
        ROW_NUMBER() OVER () AS string_id,
        normalized
    FROM (
        SELECT DISTINCT normalized
        FROM main.normalized_contexts
    ) sub;
    """))
    
    connection.execute(text("""
    CREATE TABLE main.contexts_to_normalized_strings (
    id INTEGER ,
    harmonized_name TEXT ,
    normalized_context_string_id INTEGER
    );
    """))

    connection.execute(text("""
    INSERT INTO main.contexts_to_normalized_strings (id, harmonized_name, normalized_context_string_id)
    SELECT
        nc.id,
        nc.harmonized_name,
        ncs.normalized_context_string_id
    FROM
        main.normalized_contexts nc
    JOIN main.normalized_context_strings ncs
    ON
        nc.normalized = ncs.normalized_context_string;
    """))

    connection.execute(text("""
    drop table if exists main.normalized_contexts;
    """))
    connection.execute(text("""
    drop table if exists normalized_contexts;
    """))

    transaction.commit()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [23]:
list_tables(duckdb_engine)

Tables in the current schema:
 - attributes
 - contexts_to_normalized_strings
 - harmonized_attributes_wide
 - ids
 - links
 - normalized_context_strings
 - organism
 - overview


In [24]:
df = pd.read_sql("select * from main.normalized_context_strings", duckdb_engine)

In [25]:
df

,normalized_context_string_id,normalized_context_string
0,1,not determined
1,2,arboreal habitat
2,3,antarctic cold desert
3,4,managed aquifer recharge system
4,5,water
...,...,...
165331,165332,peatland[envo_00000044]
165332,165333,tropica
165333,165334,6.6 estuary/coastal waters
165334,165335,hinderbanken


In [26]:
# Step 1: Split the 'normalized_context_string' column on the pipe character
df['normalized_context_string'] = df['normalized_context_string'].str.split('|')


In [27]:
# Step 2: Explode the DataFrame to create separate rows for each split value
df = df.explode('normalized_context_string', ignore_index=True)

In [28]:
df

,normalized_context_string_id,normalized_context_string
0,1,not determined
1,2,arboreal habitat
2,3,antarctic cold desert
3,4,managed aquifer recharge system
4,5,water
...,...,...
177894,165332,peatland[envo_00000044]
177895,165333,tropica
177896,165334,6.6 estuary/coastal waters
177897,165335,hinderbanken


In [29]:
unique_count = df['normalized_context_string'].nunique()


In [30]:
print(f"Number of unique values: {unique_count}")

Number of unique values: 164651


In [31]:
with duckdb_engine.connect() as connection:
    transaction = connection.begin()
    connection.execute(text("DROP TABLE IF EXISTS main.split_normalized_contexts;"))
    transaction.commit()

In [33]:
with duckdb_engine.connect() as connection:
    df.to_sql('split_normalized_contexts', connection, if_exists='replace', index=False)

In [34]:
with duckdb_engine.connect() as connection:
    transaction = connection.begin()
    connection.execute(text("DROP TABLE IF EXISTS main.normalized_to_split;"))
    connection.execute(text("DROP TABLE IF EXISTS main.unique_split_normalized_context_strings;"))
    connection.execute(text("DROP TABLE IF EXISTS normalized_to_split;"))
    connection.execute(text("DROP TABLE IF EXISTS unique_split_normalized_context_strings;"))
    transaction.commit()

In [35]:
with duckdb_engine.connect() as connection:
    transaction = connection.begin()
    connection.execute(text("""
    CREATE TABLE main.unique_split_normalized_context_strings (
    unique_split_normalized_context_string_id INTEGER PRIMARY KEY,
    unique_split_normalized_context_string TEXT UNIQUE
    );
    """))

    connection.execute(text("""
    INSERT INTO main.unique_split_normalized_context_strings (unique_split_normalized_context_string_id, unique_split_normalized_context_string)
    SELECT
        ROW_NUMBER() OVER () AS unique_split_normalized_context_string_id,
        normalized_context_string
    FROM (
        SELECT DISTINCT normalized_context_string
        FROM main.split_normalized_contexts
    ) sub;
    """))

    connection.execute(text("""
    CREATE TABLE main.normalized_to_split (
    normalized_context_string_id INTEGER ,
    unique_split_normalized_context_string_id INTEGER
    );
    """))

    connection.execute(text("""
    INSERT INTO main.normalized_to_split (normalized_context_string_id, unique_split_normalized_context_string_id)
    SELECT
        snc.normalized_context_string_id,
        usncs.unique_split_normalized_context_string_id
    FROM
        main.split_normalized_contexts snc
    JOIN main.unique_split_normalized_context_strings usncs
    ON
        snc.normalized_context_string = usncs.unique_split_normalized_context_string;
    """))

    transaction.commit()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [36]:
unique_split_normalized_context_strings = pd.read_sql("select * from main.unique_split_normalized_context_strings", duckdb_engine)

In [37]:
unique_split_normalized_context_strings

,unique_split_normalized_context_string_id,unique_split_normalized_context_string
0,1,increased acetate concentrations
1,2,sediment biome
2,3,orchard
3,4,rhododendron sp. 'golden belle'
4,5,quercus ilex
...,...,...
164647,164648,bto:0002135
164648,164649,bto:0002244
164649,164650,bto:0002320
164650,164651,bto:0002416


In [38]:
# Count occurrences of 'envo:' and 'envo_' for each row
unique_split_normalized_context_strings['envo_count'] = (
    unique_split_normalized_context_strings['unique_split_normalized_context_string'].str.count('envo:') +
    unique_split_normalized_context_strings['unique_split_normalized_context_string'].str.count('envo_')
)

In [39]:
unique_split_normalized_context_strings['envo_count'].value_counts()

envo_count
0.0    141771
1.0     22309
2.0       523
3.0        42
4.0         5
7.0         1
Name: count, dtype: int64

In [40]:
# Ensure the column has string values and handle non-string entries
unique_split_normalized_context_strings['unique_split_normalized_context_string'] = unique_split_normalized_context_strings['unique_split_normalized_context_string'].fillna("").astype(str)


In [41]:
# Apply the CURIE extraction function to each row
curies_list = unique_split_normalized_context_strings.apply(
    lambda row: extract_curies_from_text(
        row["unique_split_normalized_context_string"],
        row.get("unique_split_normalized_context_string_id", None)  # Pass row ID if available
    ), axis=1
).explode().dropna()

In [42]:
# Convert the list of dictionaries into a DataFrame
direct_curies_in_splits = pd.DataFrame(curies_list.tolist())

In [43]:
direct_curies_in_splits['reassembled_curie'] = direct_curies_in_splits['curie_prefix'].fillna('').str.upper() + \
                                 ":" + \
                                 direct_curies_in_splits['curie_local_id'].fillna('')

In [44]:
# Drop the specified columns
direct_curies_in_splits = direct_curies_in_splits.drop(columns=['curie_delimiter', 'curie_local_id'])

In [45]:
direct_curies_in_splits = direct_curies_in_splits.rename(columns={'row_id': 'unique_split_normalized_context_string_id'})

In [46]:
direct_curies_in_splits

,unique_split_normalized_context_string_id,curie_prefix,reassembled_curie
0,6,envo,ENVO:01001581
1,8,envo,ENVO:00000044
2,12,envo,ENVO:00005770
3,13,envo,ENVO:04000009
4,14,envo,ENVO:00000078
...,...,...,...
26863,164648,bto,BTO:0002135
26864,164649,bto,BTO:0002244
26865,164650,bto,BTO:0002320
26866,164651,bto,BTO:0002416


In [47]:
direct_curies_in_splits['curie_prefix'].value_counts()

curie_prefix
envo      21368
bto        2020
uberon     1915
foodon      441
gut         273
          ...  
lsub          1
tto           1
mat           1
ido           1
lbo           1
Name: count, Length: 67, dtype: int64

In [48]:
with duckdb_engine.connect() as connection:
    direct_curies_in_splits.to_sql('direct_curies_in_splits', connection, if_exists='replace', index=False)

In [49]:
ontologies = ['envo']

In [50]:
ontology_adapters = create_ontology_adapters(ontologies)

In [51]:
annotations_list = unique_split_normalized_context_strings.apply(
    lambda row: class_detection_by_label(
        row["unique_split_normalized_context_string"],
        ontology_adapters,
        row["unique_split_normalized_context_string_id"]
    ), axis=1
).explode().dropna()

ERROR:root:Skipping statements(subject=ENVO:00000112,predicate=oio:hasDbXref,object=<http://www.eionet.europa.eu/gemet/concept/8704>,value=None,datatype=None,language=None,); ValueError: <http://www.eionet.europa.eu/gemet/concept/8704> is not a valid URI or CURIE
ERROR:root:Skipping statements(subject=ENVO:00001996,predicate=oio:hasDbXref,object=<https://en.wikipedia.org/wiki/Acid_mine_drainage>,value=None,datatype=None,language=None,); ValueError: <https://en.wikipedia.org/wiki/Acid_mine_drainage> is not a valid URI or CURIE
ERROR:root:Skipping statements(subject=ENVO:01000225,predicate=oio:hasDbXref,object=<https://www.worldwildlife.org/biomes/tropical-and-subtropical-dry-broadleaf-forests>,value=None,datatype=None,language=None,); ValueError: <https://www.worldwildlife.org/biomes/tropical-and-subtropical-dry-broadleaf-forests> is not a valid URI or CURIE
ERROR:root:Skipping statements(subject=ENVO:01000227,predicate=oio:hasDbXref,object=<https://www.worldwildlife.org/biomes/tropical

In [52]:
# Convert the list of dictionaries into a DataFrame
curies_of_strings_of_splits = pd.DataFrame(annotations_list.tolist())

In [53]:
# Display the resulting DataFrame
curies_of_strings_of_splits

,id,subject_string,subject_start,subject_end,predicate_id,concluded_curie,object_string,is_longest_match
0,1,concentration,20,32,oio:hasExactSynonym,PATO:0000033,concentration,True
1,2,biome,10,14,rdfs:label,ENVO:00000428,biome,False
2,2,sediment,1,8,rdfs:label,ENVO:00002007,sediment,True
3,3,orchard,2,8,rdfs:label,ENVO:00000115,orchard,True
4,7,sea,1,3,rdfs:label,ENVO:00000016,sea,False
...,...,...,...,...,...,...,...,...
317146,164633,experimental forest ecosystem,1,29,rdfs:label,ENVO:03600038,experimental forest ecosystem,True
317147,164634,wet,1,3,oio:hasRelatedSynonym,ENVO:03501370,wet,True
317148,164634,wet,1,3,rdfs:label,PATO:0001823,wet,True
317149,164635,estuary,1,7,rdfs:label,ENVO:00000045,estuary,True


In [54]:
with duckdb_engine.connect() as connection:
    curies_of_strings_of_splits.to_sql('curies_of_strings_of_splits', connection, if_exists='replace', index=False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [55]:
list_tables(duckdb_engine)

Tables in the current schema:
 - attributes
 - contexts_to_normalized_strings
 - curies_of_strings_of_splits
 - direct_curies_in_splits
 - harmonized_attributes_wide
 - ids
 - links
 - normalized_context_strings
 - normalized_to_split
 - organism
 - overview
 - split_normalized_contexts
 - unique_split_normalized_context_strings


In [56]:
duckdb_engine.dispose()